# This Notebook Is for Exploring Importing Data from IEX into Apache Cassandra
## Future works will be converted into python scripts which will be more easily 'dockerized' for managing requirements and run environments
### Although Cassandra's advantages are more in enterprise-level write performance, I think its widespread use and compatibility with DFS will make it a better option than Mongo for any crazy timelines where this turns into anything

In [1]:
# Import Requirements
import requests
import csv
from bs4 import BeautifulSoup

In [2]:
#Use requests session to pull Wikipedia's S&P500 list page, this will hopefully not change too much or else this might break
session = requests.Session()
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = session.get(url)
#Pull the content (html) of the HTTP response object into BSoup and use the html parser
soup = BeautifulSoup(response.content,'html.parser')

In [3]:
#Utilize the constituents id to locate the relavant HTML info
const_table = soup.find(id='constituents')
#More parsing...
table_row = const_table.find_all('tr')[1:] #Use of list indexing removes the table header which is also a tr
#Create an empty list to receive the tickers for the S&P500 companies
sp500 = []
#Loop over BSoup result set, each 'row' being a BSoup object that can be further parsed into table data
for row in table_row:
    columns = row.find_all('td')
    #Based on the layout of the table, the second column will contain the symbols
    symbol_col = columns[1]
    sp500.append(symbol_col.text)
#Confirm the length of the list as 505
print(len(sp500))

505


In [4]:
#Write list to a csv file for durability
with open ('./sp500.csv','w',newline='') as csvfile:
    csvwriter = csv.writer(csvfile,delimiter=',')
    for ticker in sp500:
        csvwriter.writerow([ticker])

In [5]:
#Check the csv file
with open('./sp500.csv','r',newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    counter = 0
    for row in csvreader:
        counter += 1
        print(row, counter)

['MMM'] 1
['ABT'] 2
['ABBV'] 3
['ABMD'] 4
['ACN'] 5
['ATVI'] 6
['ADBE'] 7
['AMD'] 8
['AAP'] 9
['AES'] 10
['AMG'] 11
['AFL'] 12
['A'] 13
['APD'] 14
['AKAM'] 15
['ALK'] 16
['ALB'] 17
['ARE'] 18
['ALXN'] 19
['ALGN'] 20
['ALLE'] 21
['AGN'] 22
['ADS'] 23
['LNT'] 24
['ALL'] 25
['GOOGL'] 26
['GOOG'] 27
['MO'] 28
['AMZN'] 29
['AEE'] 30
['AAL'] 31
['AEP'] 32
['AXP'] 33
['AIG'] 34
['AMT'] 35
['AWK'] 36
['AMP'] 37
['ABC'] 38
['AME'] 39
['AMGN'] 40
['APH'] 41
['APC'] 42
['ADI'] 43
['ANSS'] 44
['ANTM'] 45
['AON'] 46
['AOS'] 47
['APA'] 48
['AIV'] 49
['AAPL'] 50
['AMAT'] 51
['APTV'] 52
['ADM'] 53
['ARNC'] 54
['ANET'] 55
['AJG'] 56
['AIZ'] 57
['ATO'] 58
['T'] 59
['ADSK'] 60
['ADP'] 61
['AZO'] 62
['AVB'] 63
['AVY'] 64
['BHGE'] 65
['BLL'] 66
['BAC'] 67
['BK'] 68
['BAX'] 69
['BBT'] 70
['BDX'] 71
['BRK-B'] 72
['BBY'] 73
['BIIB'] 74
['BLK'] 75
['HRB'] 76
['BA'] 77
['BKNG'] 78
['BWA'] 79
['BXP'] 80
['BSX'] 81
['BHF'] 82
['BMY'] 83
['AVGO'] 84
['BR'] 85
['BF-B'] 86
['CHRW'] 87
['COG'] 88
['CDNS'] 89
['CPB'] 

In [66]:
#Import Cassandra driver
from cassandra.cluster import Cluster
from cassandra.cqlengine.management import sync_table

In [70]:
#Establish a Cluster object that will connect to the cassandra keyspace
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('sp500')

In [67]:
#Import the ORM Model for future mapping, but I'm going to try and make this work iteratively
from cassandra.cqlengine.models import Model
from cassandra.cqlengine import columns

In [11]:
#Open the S&P500 csv and begin iterating over ticker symbols
list_of_sp500 = [] #This doesn't need to be done twice, but I'm keeping the cells modular for easier copypasta later
with open('./sp500.csv','r',newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        list_of_sp500.append(row[0]) #The reader will return an interative object so we reference the sole value

In [60]:
#Define a function that will accept a ticker string and request
# the 5 year JSON history from IEX
def GetAndReturnJSON(ticker):
    '''This function accepts a string corresponding to a stock ticker'''
    #Build out the url string from the ticker for 5 year history
    url = "https://api.iextrading.com/1.0/stock/"+ticker+"/chart/5y"
    session = requests.Session()
    response = session.get(url)
    return response.json()

#TODO -- Build the function that will pass the JSON into a Cassandra Model
# and then create that row in the S&P500 table
def InsertStockData(ticker,JSON):
    #This may be an atrocious way to do this, but in the exploratory phase
    # I'm willing to define the model within the function for right now
    class SP500(Model):
        id = columns.UUID(primary_key=True)
        ticker = columns.Text() #This one will not be returned by IEX, but will be populated while iterating
        change = columns.Float()
        changeOverTime = columns.Double()
        changePercent = columns.Float()
        close = columns.Float()
        date = columns.Text() #The IEX returns it as a date string, which would need to be processed later
        high = columns.Float()
        label = columns.Text()
        low = columns.Float()
        open = columns.Float()
        unadjustedVolume = columns.BigInt()
        volume = columns.BigInt()
        vwap = columsn.Float()
    sync_table(SP500)
    daily_stock = SP500(ticker=ticker,
         change=JSON.change,
         changeOverTime=JSON.changeOverTime,
         changePercent=JSON.changePercent,
         close=JSON.close,
         date=JSON.date,
         high=JSON.high,
         label=JSON.label,
         low=JSON.low,
         open=JSON.open,
         unadjustedVolume=JSON.unadjustedVolume,
         volume=JSON.volume,
         vwap=JSON.vwap)
    daily_stock.save()
    return(print(Hi))

In [61]:
jsontest = GetAndReturnJSON('AAPL')[0]
InsertStockData('AAPL',jsontest)

AttributeError: ResultSet object has no attribute 'UUID'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [74]:
from cassandra.cqlengine import connection
connection.setup(['127.0.0.1'],"sp500",protocol_version=3)
class SP500(Model):
    id=columns.UUID(primary_key=True)
sync_table(SP500)